# Data cleaning

## Of course, pandas!

In [65]:
import pandas as pd

## Tidy data

A dataset is said to be **tidy** if:
- Each variable forms a column
- Each row forms an observation
- Each type of observational data forms a table

One can match this concept with the third normal form from Codd's relational databases.

Some features:
- Easy for data analysts to extract different variables
- Well-suited for vectorized programming languages

There are also conventions for variables ordering:
- Fixed variables (those whose values are previously known before the experiments) comes first
- Measured variables (actually measured during the experiment) comes after
- Rows are ordered by the first (fixed) variable, then by the second (fixed), and so on.

### Data tidying

#### When column names are values, not variable names

Consider the following dataset:

In [66]:
columns = ['religion','<$10k','$10-20k','$20-30k','$30-40k','$40-50k','$50-75k']

data = [
    ['Agnostic', 27, 34, 60, 81, 76, 137],
    ['Atheist', 12, 27, 37, 52, 35, 70],
    ['Buddhist', 27, 21, 30, 34, 33, 58],
    ['Catholic', 418, 617, 732, 670, 638, 1116],
    ['Dont know/refused', 15, 14, 15, 11, 10, 35],
    ['Evangelical Prot', 575, 869, 1064, 982, 881, 1486],
    ['Hindu', 1, 9, 7, 9, 11, 34],
    ['Historically Black Prot', 228, 244, 236, 238, 197, 223],
    ['Jehovahs Witness', 20, 27, 24, 24, 21, 30],
    ['Jewish', 19, 19, 25, 25, 30, 95],
]

df = pd.DataFrame(data=data, columns=columns)

df

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


Data **melting** is the solution! We need to turn columns into rows to make the data tidy. Pandas offers the `melt` function, which requires the specification of columns that are already variables. In this case, `religion` is the only one. The others will be melted into one column for the variable and another for the values. Notice how `melt` can be used:

In [67]:
df_molten = pd.melt(df, id_vars=['religion'], var_name='income', value_name='freq')

df_molten.head(10)

,religion,income,freq
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Dont know/refused,<$10k,15
5,Evangelical Prot,<$10k,575
6,Hindu,<$10k,1
7,Historically Black Prot,<$10k,228
8,Jehovahs Witness,<$10k,20
9,Jewish,<$10k,19


#### Multiple variables stored in one column

After melting, the new column may store different values in each cell, which is not a tidy feature. To make this dataset tidy,
it is necessary to create new columns from that one, separating the semantic of the mixed values. 

Consider the dataset:

In [68]:
columns = ['country','year','m014','m1524','m65','mu','f014','f1524','f65','fu']

data = [
    ['AD',2000,0,0,0,'-',0,0,0,'-'],
    ['AE',2000,2,4,10,'-',1,3,11,'-'],
    ['AF',2000,52,228,80,'-',93,100,200,'-'],
    ['AG',2000,0,0,0,'-',0,0,0,'-'],
    ['AL',2000,13,122,1,'-',0,10,24,'-'],
    ['AM',2000,18,100,'-','-',100,12,13,'-'],
    ['AN',2000,52,20,30,'-',40,50,60,'-'],
    ['AO',2000,100,'-',111,'-',123,42,152,'-'],
    ['AR',2000,111,222,333,'-',224,552,1,'-'],
    ['AS',2000,'-','-','-','-',0,0,0,'-'],
]

df = pd.DataFrame(data=data, columns=columns)

df

,country,year,m014,m1524,m65,mu,f014,f1524,f65,fu
0,AD,2000,0,0,0,-,0,0,0,-
1,AE,2000,2,4,10,-,1,3,11,-
2,AF,2000,52,228,80,-,93,100,200,-
3,AG,2000,0,0,0,-,0,0,0,-
4,AL,2000,13,122,1,-,0,10,24,-
5,AM,2000,18,100,-,-,100,12,13,-
6,AN,2000,52,20,30,-,40,50,60,-
7,AO,2000,100,-,111,-,123,42,152,-
8,AR,2000,111,222,333,-,224,552,1,-
9,AS,2000,-,-,-,-,0,0,0,-


Notice that there are column names which are actually values (sex and ages). So, melting is the solution:

In [69]:
df_molten = pd.melt(df, id_vars=['country','year'], var_name='sex_age', value_name='cases')

df_molten.head(10)

,country,year,sex_age,cases
0,AD,2000,m014,0
1,AE,2000,m014,2
2,AF,2000,m014,52
3,AG,2000,m014,0
4,AL,2000,m014,13
5,AM,2000,m014,18
6,AN,2000,m014,52
7,AO,2000,m014,100
8,AR,2000,m014,111
9,AS,2000,m014,-


Notice, however, that column `sex_age` stores two variables, `sex` and `age`. So, this is still untidy! To overcome this, a column splitting with `str` is used:

In [70]:
df_molten['sex'] = df_molten['sex_age'].str[0]

df_molten['age_group'] = df_molten['sex_age'].str[1:-2] + '-' + df_molten['sex_age'].str[-2:]

df_molten = df_molten.drop('sex_age', axis=1)

df_molten.head(5)

,country,year,cases,sex,age_group
0,AD,2000,0,m,0-14
1,AE,2000,2,m,0-14
2,AF,2000,52,m,0-14
3,AG,2000,0,m,0-14
4,AL,2000,13,m,0-14
